In [2]:
import numpy as np
from investment_planner import calculateTransitionPropabilitiesForAllPorfolios, generateGrid

In [3]:

portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057], [0.0886, 0.1954]])
WT = np.array([90,95,100,110])
WT1 = np.array([90,100,110,120])
propabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1)
print(np.round(propabilities, 3))
   

[[[0.514 0.485 0.    0.   ]
  [0.361 0.409 0.186 0.043]
  [0.288 0.302 0.245 0.165]]

 [[0.017 0.945 0.038 0.   ]
  [0.205 0.397 0.293 0.105]
  [0.238 0.289 0.269 0.204]]

 [[0.    0.427 0.57  0.003]
  [0.099 0.32  0.375 0.205]
  [0.194 0.272 0.288 0.246]]

 [[0.    0.    0.372 0.627]
  [0.017 0.139 0.384 0.459]
  [0.127 0.232 0.311 0.33 ]]]


In [11]:
portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057], [0.0886, 0.1954]])
W0 = 30000
T = 10
infusions = [0,25000,25000,25000,25000,25000,25000,25000,25000,25000,25000]
goals = [0,0,0,0,0,0,0,0,0,0,65]
iMax = 900

grid = generateGrid(W0,T,iMax, infusions, goals, portfolios[0,0], portfolios[0,1], portfolios[2,0], portfolios[2,1])
grid = grid.T

In [23]:
k = np.array([[105,100],[110,200]])
wealthInT = np.array([90,95,100,105,110])
Wk = np.tile(wealthInT, (2,1))
ki = np.tile(k, (2, 5,1))
#np.where(Wk >= k[:,0], k[:,1], 0)
ki

array([[[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]],

       [[105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200],
        [105, 100],
        [110, 200]]])

In [ ]:
WT1 = np.array([90,95,100,105,110])
WT1P = WT1 * [0.2,0.35, 0.2 ,0.15,0.1]